# 1.Preparation

In [2]:
# Importing dependencies from transformers
from transformers import PegasusForConditionalGeneration, AutoTokenizer
# Load tokenizer 
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")

import pandas as pd
df=messages = pd.read_csv('scisumm.csv',
                           names=["text", "summary"])
df.head()
X=list(df['text'])
y=list(df['summary'])

# 2. Train Test Spliting 

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

# 3 Trainer Setup


In [4]:
from transformers import TFPegasusForConditionalGeneration, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)
with training_args.strategy.scope():
    model = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()


Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

2022-07-08 20:27:07.513572: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 393637888 exceeds 10% of free system memory.
2022-07-08 20:27:07.797736: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 393637888 exceeds 10% of free system memory.
2022-07-08 20:27:07.947964: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 393637888 exceeds 10% of free system memory.
